1. 得到dev的skeleton
2. 得到dev的schema

In [1]:
from utils.save_json_file import save_json_file
from utils.read_json_file import read_json_file
from skeleton.sql_skeleton import get_sql_skeleton,get_sql_schema
from skeleton.sql_value import extract_values
from skeleton.mapping import get_table_column_value
from utils.get_sql_schema_prompt import format_database_schema
from utils.get_full_schema_prompt import parse_schema_to_string

bird_file_path='/root/Schema-Value/data/bird/dataset_new_new/merged_with_err_gold.json'
BIRD_DATABASE='/root/Text-to-SQL/hdd/BIRD/data/sft_data_collections/bird/dev/dev_databases/'
# BIRD_DATABASE=r"E:\downloads\data\data\sft_data_collections\bird\dev\dev_databases"
data=read_json_file(bird_file_path)

new_data = []  

for item in data:  
    # full_schema = item['schema_sequence']  
    # question = item['question'] 
    pred_sql = item['sql1']  
    # err_type = item['err_type']  
    db_id = item['db_id']  

    # 假设get_sql_skeleton，get_sql_schema和extract_values是已定义的函数  
    skeleton = get_sql_skeleton(pred_sql)  
    sql_schemas = get_sql_schema(pred_sql)  
    values = extract_values(pred_sql)  
    print(pred_sql)
    print(sql_schemas)
    print(values)

    # # 假设get_table_column_value是已定义的函数，用来获取映射关系  
    # # print(pred_sql)
    schema_mapping = get_table_column_value(BIRD_DATABASE, db_id, sql_schemas, values)  

    print(schema_mapping)
    
    
    # # 假设format_database_schema是已定义的函数，用来格式化架构  
    new_schema = format_database_schema(schema_mapping)  

    print(new_schema)
    print("\n")

    
    # schema=parse_schema_to_string(full_schema)

    # 构建新的条目  
    # new_item = {  
    #     'db_id': db_id,  
    #     'question': question,  
    #     'pred_sql': pred_sql,  
    #     'err_type': err_type,  
    #     'full_schema': schema,  
    #     'new_schema': new_schema,  
    #     'skeleton': skeleton  
    # }  
    # item['schema_sequence']  = schema
    # 将新条目添加到新数据列表中  
    # new_data.append(new_item)  

# save_json_file("/root/Schema-Value/data/bird/datasets/dev.json",data)

SELECT schools.zip FROM schools INNER JOIN frpm ON schools.cdscode = frpm.cdscode WHERE frpm.'Charter School (Y/N)' = 1 AND frpm.'county name' = 'Fresno County Office of Education';
({'frpm', 'schools'}, {'Charter School (Y/N)', 'zip', 'county name', 'cdscode'}, {'Fresno County Office of Education', 'Charter School (Y/N)', '1', 'county name'})
['county name = Fresno County Office of Education']
{'frpm:county name': ['(Fresno County Office of Education)'], 'frpm:charter school (y/n)': [], 'schools:zip': [], 'frpm:cdscode': [], 'schools:cdscode': []}
table frpm, columns = [`county name` (values: 'Fresno County Office of Education'), `charter school (y/n)`, `cdscode`]; table schools, columns = [`zip`, `cdscode`]


SELECT COUNT(*) FROM schools INNER JOIN satscores ON schools.cdscode = satscores.cds WHERE satscores.avgscrmath > 400 AND schools.virtual = 'F';
({'satscores', 'schools'}, {'virtual', 'avgscrmath', 'cdscode', 'cds'}, {'400', 'F'})
['virtual = F']
{'schools:virtual': ['(F)'], 'sa

In [4]:
import os  
import sqlite3  

def get_all_columns_for_db(database_path, db_id, tables):  
    # Construct the full path to the database  
    database_name = os.path.join(database_path, db_id, f"{db_id}.sqlite")  
    
    # Set to hold unique column names across all specified tables  
    columns = set()  

    # Connect to the SQLite database  
    try:  
        connection = sqlite3.connect(database_name)  
        cursor = connection.cursor()  

        for table in tables:  
            # Execute PRAGMA query to get column info for each table  
            cursor.execute(f"PRAGMA table_info('{table}');")  
            column_info = cursor.fetchall()  

            # Extract column names and add them to the set  
            for column in column_info:  
                columns.add(column[1])  # assuming index 1 is for column names  

    except sqlite3.Error as e:  
        print(f"An error occurred: {e}")  

    finally:  
        # Ensure the connection is closed  
        if connection:  
            connection.close()  

    return columns  

# Example usage:  
BIRD_DATABASE='/root/Text-to-SQL/hdd/BIRD/data/sft_data_collections/bird/dev/dev_databases/'
db_id = "california_schools"  
tables = ["frpm", "schools"]  
column_data = get_all_columns_for_db(BIRD_DATABASE, db_id, tables)  
print(column_data)  # Output might look like {'Enrollment (Ages 5-17)', 'Enrollment (K-12)', 'school_code'}

{'Free Meal Count (K-12)', 'EILCode', 'ClosedDate', 'District Name', 'AdmEmail1', 'Low Grade', 'MailZip', 'School Code', 'Ext', 'Enrollment (Ages 5-17)', 'NCESSchool', 'County', 'GSoffered', 'StreetAbr', 'Free Meal Count (Ages 5-17)', 'AdmFName1', 'NSLP Provision Status', 'IRC', 'Charter Funding Type', 'Phone', 'DOCType', 'MailStreet', 'NCESDist', 'District', 'Charter School (Y/N)', 'Academic Year', 'Longitude', 'Charter School Number', 'County Name', 'Percent (%) Eligible Free (K-12)', 'State', 'EILName', 'Virtual', 'MailCity', 'CharterNum', 'Percent (%) Eligible FRPM (Ages 5-17)', 'LastUpdate', 'Latitude', 'Percent (%) Eligible FRPM (K-12)', 'Percent (%) Eligible Free (Ages 5-17)', 'FundingType', 'Charter', 'OpenDate', 'Enrollment (K-12)', 'AdmFName2', 'AdmFName3', 'District Code', 'Zip', 'SOC', 'AdmLName2', 'StatusType', 'EdOpsName', 'High Grade', 'School Name', 'CDSCode', 'MailStrAbr', 'EdOpsCode', 'MailState', 'Street', 'School Type', 'District Type', 'Website', 'Educational Optio

In [4]:
# 提取values
values0 = ['virtual = F','c=d']
values1 = ['a = b', 'c=d']

# 合并并去重values
values = list(set(values0 + values1))  # 去重后的values

# 获取sql_schemas
sql_schemas0 = ({'frpm'}, {'Enrollment (Ages 5-17)', 'Enrollment (K-12)', 'school_code'}, {'500'})
sql_schemas1 = ({'satscores', 'schools'}, {'virtual', 'avgscrmath', 'cdscode', 'cds'}, {'400', 'F'})

# 合并表、列和值，分别处理每个部分
tables = set(sql_schemas0[0]).union(sql_schemas1[0])  # 合并表
columns = set(sql_schemas0[1]).union(sql_schemas1[1])  # 合并列
values = set(sql_schemas0[2]).union(sql_schemas1[2])  # 合并值

# 最终结果以元组形式返回，保持原始格式
sql_schemas = (tables, columns, values)


values, sql_schemas


({'400', '500', 'F'},
 ({'frpm', 'satscores', 'schools'},
  {'Enrollment (Ages 5-17)',
   'Enrollment (K-12)',
   'avgscrmath',
   'cds',
   'cdscode',
   'school_code',
   'virtual'},
  {'400', '500', 'F'}))

In [1]:
from utils.save_json_file import save_json_file
from utils.read_json_file import read_json_file
from skeleton.sql_skeleton import get_sql_skeleton,get_sql_schema
from skeleton.sql_value import extract_values
from skeleton.mapping import get_table_column_value
from utils.get_sql_schema_prompt import format_database_schema
from utils.get_full_schema_prompt import parse_schema_to_string

bird_file_path='/root/Schema-Value/data/bird/dataset/bird_dev_test.json'
# BIRD_DATABASE='/root/Text-to-SQL/hdd/BIRD/data/sft_data_collections/bird/dev/dev_databases/'
BIRD_DATABASE="/root/Text-to-SQL/dataset/spider/database"

data=read_json_file(bird_file_path)

new_data = []  

for item in data:  
    full_schema = item['full_schema']  
    question = item['question'] 
    pred_sql = item['pred_sql']  
    err_type = item['err_type']  
    db_id = item['db_id']  

    # 假设get_sql_skeleton，get_sql_schema和extract_values是已定义的函数  
    skeleton = get_sql_skeleton(pred_sql)  
    sql_schemas = get_sql_schema(pred_sql)  
    values = extract_values(pred_sql)  

    print("sql_schemas:",sql_schemas)
    print("values:",values)

    # 假设get_table_column_value是已定义的函数，用来获取映射关系  
    # print(pred_sql)
    schema_mapping = get_table_column_value(BIRD_DATABASE, db_id, sql_schemas, values)  
    print("schema_mapping:",schema_mapping)

    # 假设format_database_schema是已定义的函数，用来格式化架构  
    new_schema = format_database_schema(schema_mapping) 
    print(new_schema) 
#     schema=parse_schema_to_string(full_schema)

#     # 构建新的条目  
#     new_item = {  
#         'db_id': db_id,  
#         'question': question,  
#         'pred_sql': pred_sql,  
#         'err_type': err_type,  
#         'full_schema': schema,  
#         'new_schema': new_schema,  
#         'skeleton': skeleton  
#     }  222

#     # 将新条目添加到新数据列表中  
#     new_data.append(new_item)  

# save_json_file(r"G:\IDEA\schema-value\data\bird\datasets\dev_process.json",new_data)

sql_schemas: ({'singer'}, set(), set())
values: []
schema_mapping: {'singer:*': []}
table singer, columns = [*]
sql_schemas: ({'cartoon', 'tv_channel'}, {'channel', 'id', 'directed_by', 'Ben Jones', 'package_option'}, set())
values: ['directed_by = Ben Jones']
column: directed_by
value: Ben Jones
columns: {'channel', 'id', 'directed_by', 'Ben Jones', 'package_option'}
column_to_table_mapping: {'id': ['TV_Channel', 'TV_series', 'Cartoon'], 'series_name': ['TV_Channel', 'TV_series'], 'country': ['TV_Channel'], 'language': ['TV_Channel'], 'content': ['TV_Channel'], 'pixel_aspect_ratio_par': ['TV_Channel'], 'hight_definition_tv': ['TV_Channel'], 'pay_per_view_ppv': ['TV_Channel'], 'package_option': ['TV_Channel'], 'episode': ['TV_series'], 'air_date': ['TV_series'], 'rating': ['TV_series'], 'share': ['TV_series'], '18_49_rating_share': ['TV_series'], 'viewers_m': ['TV_series'], 'weekly_rank': ['TV_series'], 'channel': ['TV_series', 'Cartoon'], 'title': ['Cartoon'], 'directed_by': ['Cartoon

bird

In [1]:
from utils.save_json_file import save_json_file
from utils.read_json_file import read_json_file
from skeleton.sql_skeleton import get_sql_skeleton,get_sql_schema
from skeleton.sql_value import extract_values
from skeleton.mapping import get_table_column_value
from utils.get_sql_schema_prompt import format_database_schema
from utils.get_full_schema_prompt import parse_schema_to_string

bird_file_path="/root/Schema-Value/data/bird/dev_1412.json"
BIRD_DATABASE='/root/Text-to-SQL/hdd/BIRD/data/sft_data_collections/bird/dev/dev_databases/'
data=read_json_file(bird_file_path)

new_data = []  

for item in data:  
    full_schema = item['full_schema']  
    question = item['question'] 
    pred_sql = item['pred_sql']  
    err_type = item['err_type']  
    db_id = item['db_id']  

    # 假设get_sql_skeleton，get_sql_schema和extract_values是已定义的函数  
    skeleton = get_sql_skeleton(pred_sql)  
    sql_schemas = get_sql_schema(pred_sql)  
    values = extract_values(pred_sql)  

    # 假设get_table_column_value是已定义的函数，用来获取映射关系  
    # print(pred_sql)
    schema_mapping = get_table_column_value(BIRD_DATABASE, db_id, sql_schemas, values)  

    # 假设format_database_schema是已定义的函数，用来格式化架构  
    new_schema = format_database_schema(schema_mapping)  
    # schema=parse_schema_to_string(full_schema)

    # 构建新的条目  
    new_item = {  
        'db_id': db_id,  
        'question': question,  
        'pred_sql': pred_sql,  
        'err_type': err_type,  
        'full_schema': full_schema,  
        'new_schema': new_schema,  
        'skeleton': skeleton  
    }  

    # 将新条目添加到新数据列表中  
    new_data.append(new_item)  

save_json_file("/root/Schema-Value/data/bird/dev_1412_new_schema.json",new_data)

In [7]:
from utils.save_json_file import save_json_file
from utils.read_json_file import read_json_file
from skeleton.sql_skeleton import get_sql_skeleton,get_sql_schema
from skeleton.sql_value import extract_values
from skeleton.mapping import get_table_column_value
from utils.get_sql_schema_prompt import format_database_schema
from utils.get_full_schema_prompt import parse_schema_to_string

bird_file_path="/root/Schema-Value/data/bird/dataset_new_new/foreign_data.json"
BIRD_DATABASE='/root/Text-to-SQL/hdd/BIRD/data/sft_data_collections/bird/dev/dev_databases/'
data=read_json_file(bird_file_path)

new_data = []  

for item in data:  
    full_schema = item['schema_sequence']  
    question = item['question'] + " " + item['evidence'] 
    pred_sql = item['err_pred']  
    err_type = item['err_type']  
    db_id = item['db_id']  

    # 假设get_sql_skeleton，get_sql_schema和extract_values是已定义的函数  
    skeleton = get_sql_skeleton(pred_sql)  
    sql_schemas = get_sql_schema(pred_sql)  
    values = extract_values(pred_sql)  

    # 假设get_table_column_value是已定义的函数，用来获取映射关系  
    # print(pred_sql)
    schema_mapping = get_table_column_value(BIRD_DATABASE, db_id, sql_schemas, values)  

    # 假设format_database_schema是已定义的函数，用来格式化架构  
    new_schema = format_database_schema(schema_mapping)  
    schema=parse_schema_to_string(full_schema)

    # 构建新的条目  
    new_item = {  
        'db_id': db_id,  
        'question': question,  
        'pred_sql': pred_sql,  
        'err_type': err_type,  
        'full_schema': schema,  
        'new_schema': new_schema,  
        'skeleton': skeleton  
    }  

    # 将新条目添加到新数据列表中  
    new_data.append(new_item)  

save_json_file("/root/Schema-Value/data/bird/dataset_new_new/foreign_process.json",new_data)

spider

In [1]:
from utils.save_json_file import save_json_file
from utils.read_json_file import read_json_file
from skeleton.sql_skeleton import get_sql_skeleton,get_sql_schema
from skeleton.sql_value import extract_values
from skeleton.mapping import get_table_column_value
from utils.get_prompt_spider import create_prompt
from utils.get_sql_schema_prompt import format_database_schema

bird_file_path="/root/Schema-Value/data/spider/dataset_new/foreign_data.json"
SPIDER_DATABASE="/root/Text-to-SQL/dataset/spider/database"
data=read_json_file(bird_file_path)


new_data = []  

for item in data:  
    full_schema = create_prompt(item)
    question = item['question'] 
    pred_sql = item['err_pred']  
    err_type = item['err_type']  
    db_id = item['db_id']  
    id = item['id']  

    # 假设get_sql_skeleton，get_sql_schema和extract_values是已定义的函数  
    skeleton = get_sql_skeleton(pred_sql)  
    sql_schemas = get_sql_schema(pred_sql)  
    values = extract_values(pred_sql)  

    # 假设get_table_column_value是已定义的函数，用来获取映射关系  
    # print(pred_sql)
    schema_mapping = get_table_column_value(SPIDER_DATABASE, db_id, sql_schemas, values)  

    # 假设format_database_schema是已定义的函数，用来格式化架构  
    new_schema = format_database_schema(schema_mapping)  

    # 构建新的条目  
    new_item = {  
        'id': id,
        'db_id': db_id,  
        'question': question,  
        'pred_sql': pred_sql,  
        'err_type': err_type,  
        'full_schema': full_schema,  
        'new_schema': new_schema,  
        'skeleton': skeleton  
    }  

    # 将新条目添加到新数据列表中  
    new_data.append(new_item)  

save_json_file("/root/Schema-Value/data/spider/dataset_new/foreign_processs.json",new_data)

/root/anaconda3/envs/env_llama_factory/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from utils.save_json_file import save_json_file
from utils.read_json_file import read_json_file
from skeleton.sql_skeleton import get_sql_skeleton,get_sql_schema
from skeleton.sql_value import extract_values
from skeleton.mapping import get_table_column_value
from utils.get_prompt_spider import create_prompt
from utils.get_sql_schema_prompt import format_database_schema

bird_file_path="/root/Schema-Value/data/spider/dev_889.json"
SPIDER_DATABASE="/root/Text-to-SQL/dataset/spider/database"
data=read_json_file(bird_file_path)


new_data = []  

for item in data:  
    full_schema = create_prompt(item)
    question = item['question'] 
    pred_sql = item['pred_sql']  
    err_type = item['err_type']  
    db_id = item['db_id']  
    # id = item['id']  

    # 假设get_sql_skeleton，get_sql_schema和extract_values是已定义的函数  
    skeleton = get_sql_skeleton(pred_sql)  
    sql_schemas = get_sql_schema(pred_sql)  
    values = extract_values(pred_sql)  

    # 假设get_table_column_value是已定义的函数，用来获取映射关系  
    # print(pred_sql)
    schema_mapping = get_table_column_value(SPIDER_DATABASE, db_id, sql_schemas, values)  

    # 假设format_database_schema是已定义的函数，用来格式化架构  
    new_schema = format_database_schema(schema_mapping)  

    # 构建新的条目  
    new_item = {  
        # 'id': id,
        'db_id': db_id,  
        'question': question,  
        'pred_sql': pred_sql,  
        'err_type': err_type,  
        'full_schema': full_schema,  
        'new_schema': new_schema,  
        'skeleton': skeleton  
    }  

    # 将新条目添加到新数据列表中  
    new_data.append(new_item)  

save_json_file("/root/Schema-Value/data/spider/dev_889_new_schema.json",new_data)

/root/anaconda3/envs/env_llama_factory/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


question + schema-full + SQL-skeleton 生成SQL1

In [2]:
from utils.get_response import get_gpt_response
from utils.prompt import question_schema_skeleton_instruction,question_schema_skeleton_inputs
from utils.read_json_file import read_json_file
from utils.save_json_file import save_json_file

i=0
data = read_json_file(r"C:\Users\hyjing\Desktop\sql文件\dev_process.json")
for item in data:
    if i>10:
        break
    question=item['question']
    full_schema=item['full_schema']
    skeleton=item['skeleton']
    inputs=question_schema_skeleton_inputs.format(
        question=question,
        schema_full=full_schema,
        skeleton=skeleton
    )
    print(inputs)
    sql=get_gpt_response(question_schema_skeleton_instruction,inputs)
    print(sql)
    i=i+1


Questino: What is the highest eligible free rate for K-12 students in the schools in Alameda County? Eligible free rate for K-12 = 'Free Meal Count (K-12)' / 'Enrollment (K-12)'
Schema: database schema :
table frpm , columns = [ frpm.cdscode ( text | primary key | values : 01100170109835 , 01100170112607 ) , frpm.'academic year' ( text | values : 2014-2015 ) , frpm.'county code' ( text | values : 01 , 02 ) , frpm.'district code' ( integer | values : 10017 , 31609 ) , frpm.'school code' ( text | values : 0109835 , 0112607 ) , frpm.'county name' ( text | values : Alameda , Alpine ) , frpm.'district name' ( text ) , frpm.'school name' ( text | values : FAME Public Charter ) , frpm.'district type' ( text | values : State Special Schools ) , frpm.'school type' ( text | values : K-12 Schools (Public) , High Schools (Public) ) , frpm.'educational option type' ( text | values : Traditional , Juvenile Court School ) , frpm.'nslp provision status' ( text | values : Breakfast Provision 2 , Provis

In [40]:
import os  
import json  

def merge_json_files(folder_path, output_file):  
    """  
    合并指定文件夹内的多个 JSON 文件，将结果平面化并写入一个新的 JSON 文件。  
    
    参数:  
    folder_path (str): 包含要合并的 JSON 文件的文件夹路径。  
    output_file (str): 合并后的 JSON 文件路径。  
    """  
    filenames = [f"sql1-{i}.json" for i in range(1, 8)]  
    merged_data = []  

    # 读取并平面化合并 JSON 文件的内容  
    for filename in filenames:  
        file_path = os.path.join(folder_path, filename)  
        try:  
            with open(file_path, 'r', encoding='utf-8') as f:  
                data = json.load(f)  
                if isinstance(data, list):  
                    merged_data.extend(data)  
                else:  
                    print(f"警告: 文件 {file_path} 不包含列表，略过该文件！")  
        except FileNotFoundError:  
            print(f"文件 {file_path} 未找到！")  
        except json.JSONDecodeError:  
            print(f"文件 {file_path} 不是有效的 JSON 文件！")  

    # 写入合并后的数据到新的 JSON 文件  
    with open(output_file, 'w', encoding='utf-8') as f:  
        json.dump(merged_data, f, ensure_ascii=False, indent=4)  

if __name__ == "__main__":  
    folder_path = r"G:\IDEA\schema-value\data\bird\datasets"  
    output_file = os.path.join(folder_path, "merged_sql1.json")  
    merge_json_files(folder_path, output_file)

question + schema-new 生成SQL2

对比SQL1和predSQL的结果/schema
对比SQL2和predSQL的结果/skeleton

In [16]:

import json
from sklearn.metrics import  classification_report
import numpy as np

def combine_error_types(file1_path):
    # 读取JSON文件
    with open(file1_path, 'r', encoding='utf-8') as file1:
        data1 = json.load(file1)


    # 假设两个文件的数据是一个列表，每个元素对应一个SQL记录
    combined_data = []
    y_true = []
    y_pred = []
    true_err_count=0
    pred_err_count=0
    for item1 in data1:
        # 获取两个文件中的new_err_type
        new_err_type1 = item1.get('new_err_type', '')
        true_err_type = item1.get('err_type','')

        # 判断combine_err_type的值
        if new_err_type1 == 'empty':
            pred_err_count+=1
            if true_err_type == 'result':
                true_err_count+=1
            else:
                print(item1.get('question', ''))

        

    print("预测为错误的：",pred_err_count)
    print("实际为错误的：",true_err_count)
    print("precision:", true_err_count*1.0 / pred_err_count)
    
# 示例调用
file1_path = '/root/Schema-Value/data/bird/dataset_new_new_new/bird_dev.json'
combine_error_types(file1_path)





预测为错误的： 47
实际为错误的： 47
precision: 1.0


In [12]:
import re

def extract_values(pred_sql):
    # 更新正则表达式，处理连续单引号的情况
    pattern = r"(\w+)\s*(=|!=)\s*(?:\'([^\']*(?:\'\'[^\']*)*)\'|\"([^\"]*)\")"
    matches = re.findall(pattern, pred_sql)

    # 格式化输出
    results = []
    for match in matches:
        field_name = match[0]
        operator = match[1]
        value = match[2] if match[2] else match[3]  # 优先选择单引号内的值
        # 替换连续单引号（''）为单个引号（'）
        # value = value.replace("''", "'")
        results.append(f"{field_name} = {value}")

    return results

# 测试 SQL 示例
sql_query = "SELECT count(member.member_id) FROM member INNER JOIN attendance ON member.'member_id' = attendance.'link_to_member' INNER JOIN event ON attendance.link_to_event = event.event_id WHERE event.event_name = 'Women''s Soccer' AND member.t_shirt_size = 'Medium'"

print(extract_values(sql_query))


["event_name = Women''s Soccer", 't_shirt_size = Medium']


sql1和sql2都检测为是错误的

bird

In [4]:
import json
from sklearn.metrics import  classification_report
import numpy as np

def combine_error_types(file1_path, file2_path):
    # 读取JSON文件
    with open(file1_path, 'r', encoding='utf-8') as file1, open(file2_path, 'r', encoding='utf-8') as file2:
        data1 = json.load(file1)
        data2 = json.load(file2)


    # 假设两个文件的数据是一个列表，每个元素对应一个SQL记录
    combined_data = []
    y_true = []
    y_pred = []
    true_err_count=0
    pred_err_count=0
    for item1, item2 in zip(data1, data2):
        # 获取两个文件中的new_err_type
        new_err_type1 = item1.get('new_err_type', '')
        new_err_type2 = item2.get('new_err_type', '')
        true_err_type = item1.get('err_type','')

        # 判断combine_err_type的值
        if new_err_type1 != 'correct' and new_err_type2 != 'correct':
            pred_err_count+=1
            if true_err_type != 'correct':
                true_err_count+=1

        true_label = 0 if true_err_type != "correct" else 1
        # 根据 new_err_type 确定 predicted label
        pred_label = 0 if new_err_type1 != "correct" and new_err_type2!="correct" else 1

        # 仅在不是超时的情况下记录
        # if item['new_err_type'] != "timeout":
        y_true.append(true_label)
        y_pred.append(pred_label)

    print("预测为错误的：",pred_err_count)
    print("实际为错误的：",true_err_count)
    print("precision:", true_err_count*1.0 / pred_err_count)
    # pred_labels = np.array([0]*259)
    # true_labels = np.array([0]*197 + [1]*62)
    # y_true += true_labels.tolist()
    # y_pred += pred_labels.tolist()

    if y_true and y_pred:

        report = classification_report(y_true, y_pred, target_names=["incorrect", "correct"])
        print(report)
        report_dict = classification_report(y_true, y_pred, target_names=["incorrect", "correct"], output_dict=True)  

        # 格式化和打印报告  
        print("Classification Report:")  
        for label, metrics in report_dict.items():  
            if isinstance(metrics, dict):  # 表示这是类别级别的统计  
                print(f"{label}:")  
                for metric, value in metrics.items():  
                    print(f"  {metric}: {value:.3f}")  
            else:  # 针对"accuracy"这个单值指标  
                print(f"{label}: {metrics:.3f}")
# 示例调用
file1_path = '/root/Schema-Value/data/bird/dataset_new/llama/sql1-result-gpt.json'
file2_path = '/root/Schema-Value/data/bird/dataset_new_new/deepseek/sql2-1-result.json'
combine_error_types(file1_path, file2_path)


预测为错误的： 411
实际为错误的： 311
precision: 0.7566909975669099
              precision    recall  f1-score   support

   incorrect       0.76      0.62      0.68       501
     correct       0.78      0.87      0.82       774

    accuracy                           0.77      1275
   macro avg       0.77      0.75      0.75      1275
weighted avg       0.77      0.77      0.77      1275

Classification Report:
incorrect:
  precision: 0.757
  recall: 0.621
  f1-score: 0.682
  support: 501.000
correct:
  precision: 0.780
  recall: 0.871
  f1-score: 0.823
  support: 774.000
accuracy: 0.773
macro avg:
  precision: 0.768
  recall: 0.746
  f1-score: 0.752
  support: 1275.000
weighted avg:
  precision: 0.771
  recall: 0.773
  f1-score: 0.768
  support: 1275.000


In [5]:
# 有一个错误的则认为是错误的

import json
from sklearn.metrics import  classification_report
import numpy as np

def combine_error_types(file1_path, file2_path):
    # 读取JSON文件
    with open(file1_path, 'r', encoding='utf-8') as file1, open(file2_path, 'r', encoding='utf-8') as file2:
        data1 = json.load(file1)
        data2 = json.load(file2)


    # 假设两个文件的数据是一个列表，每个元素对应一个SQL记录
    combined_data = []
    y_true = []
    y_pred = []
    true_err_count=0
    pred_err_count=0
    for item1, item2 in zip(data1, data2):
        # 获取两个文件中的new_err_type
        new_err_type1 = item1.get('new_err_type', '')
        new_err_type2 = item2.get('new_err_type', '')
        true_err_type = item1.get('err_type','')

        # 判断combine_err_type的值
        if new_err_type1 != 'correct' or new_err_type2 != 'correct':
            pred_err_count+=1
            if true_err_type != 'correct':
                true_err_count+=1

        true_label = 0 if true_err_type != "correct" else 1
        # 根据 new_err_type 确定 predicted label
        pred_label = 0 if new_err_type1 != "correct" or new_err_type2!="correct" else 1

        # 仅在不是超时的情况下记录
        # if item['new_err_type'] != "timeout":
        y_true.append(true_label)
        y_pred.append(pred_label)

    print("预测为错误的：",pred_err_count)
    print("实际为错误的：",true_err_count)
    print("precision:", true_err_count*1.0 / pred_err_count)
    # pred_labels = np.array([0]*259)
    # true_labels = np.array([0]*197 + [1]*62)
    # y_true += true_labels.tolist()
    # y_pred += pred_labels.tolist()

    if y_true and y_pred:

        report = classification_report(y_true, y_pred, target_names=["incorrect", "correct"])
        print(report)
        report_dict = classification_report(y_true, y_pred, target_names=["incorrect", "correct"], output_dict=True)  

        # 格式化和打印报告  
        print("Classification Report:")  
        for label, metrics in report_dict.items():  
            if isinstance(metrics, dict):  # 表示这是类别级别的统计  
                print(f"{label}:")  
                for metric, value in metrics.items():  
                    print(f"  {metric}: {value:.3f}")  
            else:  # 针对"accuracy"这个单值指标  
                print(f"{label}: {metrics:.3f}")
# 示例调用
file1_path = '/root/Schema-Value/data/bird/dataset_new/llama/sql1-result-gpt.json'
file2_path = '/root/Schema-Value/data/bird/dataset_new_new/deepseek/sql2-1-result.json'
combine_error_types(file1_path, file2_path)


预测为错误的： 712
实际为错误的： 424
precision: 0.5955056179775281
              precision    recall  f1-score   support

   incorrect       0.60      0.85      0.70       501
     correct       0.86      0.63      0.73       774

    accuracy                           0.71      1275
   macro avg       0.73      0.74      0.71      1275
weighted avg       0.76      0.71      0.72      1275

Classification Report:
incorrect:
  precision: 0.596
  recall: 0.846
  f1-score: 0.699
  support: 501.000
correct:
  precision: 0.863
  recall: 0.628
  f1-score: 0.727
  support: 774.000
accuracy: 0.714
macro avg:
  precision: 0.729
  recall: 0.737
  f1-score: 0.713
  support: 1275.000
weighted avg:
  precision: 0.758
  recall: 0.714
  f1-score: 0.716
  support: 1275.000


spider sql单
+system+foreign+empty

In [9]:
import json
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
import numpy as np

def combine_error_types(file1_path):
    # 读取JSON文件
    with open(file1_path, 'r', encoding='utf-8') as file1:
        data1 = json.load(file1)


    # 假设两个文件的数据是一个列表，每个元素对应一个SQL记录
    combined_data = []
    y_true = []
    y_pred = []
    true_err_count=0
    pred_err_count=0
    for item1 in data1:
        # 获取两个文件中的new_err_type
        new_err_type1 = item1.get('new_err_type', '')
        true_err_type = item1.get('err_type','')

        # 判断combine_err_type的值
        if new_err_type1 != 'correct':
            pred_err_count+=1
            if true_err_type != 'correct':
                true_err_count+=1

        true_label = 0 if true_err_type != "correct" else 1
        # 根据 new_err_type 确定 predicted label
        pred_label = 0 if new_err_type1 != "correct"  else 1

        # 仅在不是超时的情况下记录
        # if item['new_err_type'] != "timeout":
        y_true.append(true_label)
        y_pred.append(pred_label)

    print("预测为错误的：",pred_err_count)
    print("实际为错误的：",true_err_count)
    print("precision:", true_err_count*1.0 / pred_err_count)
    # pred_labels = np.array([0]*175)
    pred_labels = np.array([0]*127 + [1]*48)
    true_labels = np.array([0]*127 + [1]*48)
    # true_labels = np.array([0]*175)
    y_true += true_labels.tolist()
    y_pred += pred_labels.tolist()

    if y_true and y_pred:

        report = classification_report(y_true, y_pred, target_names=["incorrect", "correct"])
        print(report)
        report_dict = classification_report(y_true, y_pred, target_names=["incorrect", "correct"], output_dict=True)  

        # 格式化和打印报告  
        print("Classification Report:")  
        for label, metrics in report_dict.items():  
            if isinstance(metrics, dict):  # 表示这是类别级别的统计  
                print(f"{label}:")  
                for metric, value in metrics.items():  
                    print(f"  {metric}: {value:.3f}")  
            else:  # 针对"accuracy"这个单值指标  
                print(f"{label}: {metrics:.3f}")
# 示例调用
# file1_path = '/root/Schema-Value/data/bird/dataset_new/llama/sql1-result-gpt.json'

# file1_path = '/root/Schema-Value/data/spider/dataset_new/gpt/sql1-2-result.json'
combine_error_types(file1_path)


预测为错误的： 170
实际为错误的： 79
precision: 0.4647058823529412
              precision    recall  f1-score   support

   incorrect       0.69      0.81      0.75       254
     correct       0.93      0.88      0.91       780

    accuracy                           0.87      1034
   macro avg       0.81      0.85      0.83      1034
weighted avg       0.88      0.87      0.87      1034

Classification Report:
incorrect:
  precision: 0.694
  recall: 0.811
  f1-score: 0.748
  support: 254.000
correct:
  precision: 0.935
  recall: 0.883
  f1-score: 0.908
  support: 780.000
accuracy: 0.866
macro avg:
  precision: 0.814
  recall: 0.847
  f1-score: 0.828
  support: 1034.000
weighted avg:
  precision: 0.876
  recall: 0.866
  f1-score: 0.869
  support: 1034.000


spider 交集

In [4]:
import json
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
import numpy as np

def combine_error_types(file1_path, file2_path):
    # 读取JSON文件
    with open(file1_path, 'r', encoding='utf-8') as file1, open(file2_path, 'r', encoding='utf-8') as file2:
        data1 = json.load(file1)
        data2 = json.load(file2)


    # 假设两个文件的数据是一个列表，每个元素对应一个SQL记录
    combined_data = []
    y_true = []
    y_pred = []
    true_err_count=0
    pred_err_count=0
    for item1, item2 in zip(data1, data2):
        # 获取两个文件中的new_err_type
        new_err_type1 = item1.get('new_err_type', '')
        new_err_type2 = item2.get('new_err_type', '')
        true_err_type = item1.get('err_type','')

        # 判断combine_err_type的值
        if new_err_type1 != 'correct' and new_err_type2 != 'correct':
            pred_err_count+=1
            if true_err_type != 'correct':
                true_err_count+=1

        true_label = 0 if true_err_type != "correct" else 1
        # 根据 new_err_type 确定 predicted label
        pred_label = 0 if new_err_type1 != "correct" and new_err_type2!="correct" else 1

        # 仅在不是超时的情况下记录
        # if item['new_err_type'] != "timeout":
        y_true.append(true_label)
        y_pred.append(pred_label)

    print("预测为错误的：",pred_err_count)
    print("实际为错误的：",true_err_count)
    print("precision:", true_err_count*1.0 / pred_err_count)
    pred_labels = np.array([0]*127 + [1]*48)
    true_labels = np.array([0]*127 + [1]*48)
    y_true += true_labels.tolist()
    y_pred += pred_labels.tolist()

    if y_true and y_pred:

        report = classification_report(y_true, y_pred, target_names=["incorrect", "correct"])
        print(report)
        report_dict = classification_report(y_true, y_pred, target_names=["incorrect", "correct"], output_dict=True)  

        # 格式化和打印报告  
        print("Classification Report:")  
        for label, metrics in report_dict.items():  
            if isinstance(metrics, dict):  # 表示这是类别级别的统计  
                print(f"{label}:")  
                for metric, value in metrics.items():  
                    print(f"  {metric}: {value:.3f}")  
            else:  # 针对"accuracy"这个单值指标  
                print(f"{label}: {metrics:.3f}")
# 示例调用
file1_path = '/root/Schema-Value/data/spider/dataset_new/gpt/sql1-2-result.json'
file2_path = '/root/Schema-Value/data/spider/dataset_new/gpt/sql2-1-result.json'
combine_error_types(file1_path, file2_path)


预测为错误的： 134
实际为错误的： 72
precision: 0.5373134328358209
              precision    recall  f1-score   support

   incorrect       0.76      0.78      0.77       254
     correct       0.93      0.92      0.92       780

    accuracy                           0.89      1034
   macro avg       0.85      0.85      0.85      1034
weighted avg       0.89      0.89      0.89      1034

Classification Report:
incorrect:
  precision: 0.762
  recall: 0.783
  f1-score: 0.773
  support: 254.000
correct:
  precision: 0.929
  recall: 0.921
  f1-score: 0.925
  support: 780.000
accuracy: 0.887
macro avg:
  precision: 0.846
  recall: 0.852
  f1-score: 0.849
  support: 1034.000
weighted avg:
  precision: 0.888
  recall: 0.887
  f1-score: 0.887
  support: 1034.000


In [19]:
import json
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

def combine_error_types(file1_path, file2_path):
    # 读取JSON文件
    with open(file1_path, 'r', encoding='utf-8') as file1, open(file2_path, 'r', encoding='utf-8') as file2:
        data1 = json.load(file1)
        data2 = json.load(file2)


    # 假设两个文件的数据是一个列表，每个元素对应一个SQL记录
    combined_data = []
    y_true = []
    y_pred = []
    true_err_count=0
    pred_err_count=0
    for item1, item2 in zip(data1, data2):
        # 获取两个文件中的new_err_type
        new_err_type1 = item1.get('new_err_type', '')
        new_err_type2 = item2.get('new_err_type', '')
        true_err_type = item1.get('err_type','')

        # 判断combine_err_type的值
        if new_err_type1 != 'correct' or new_err_type2 != 'correct':
            pred_err_count+=1
            if true_err_type != 'correct':
                true_err_count+=1

        true_label = 0 if true_err_type != "correct" else 1
        # 根据 new_err_type 确定 predicted label
        pred_label = 0 if new_err_type1 != "correct" or new_err_type2!="correct" else 1

        # 仅在不是超时的情况下记录
        # if item['new_err_type'] != "timeout":
        y_true.append(true_label)
        y_pred.append(pred_label)

    print("预测为错误的：",pred_err_count)
    print("实际为错误的：",true_err_count)
    print("precision:", true_err_count*1.0 / pred_err_count)

    if y_true and y_pred:

        report = classification_report(y_true, y_pred, target_names=["incorrect", "correct"])
        print(report)
        report_dict = classification_report(y_true, y_pred, target_names=["incorrect", "correct"], output_dict=True)  

        # 格式化和打印报告  
        print("Classification Report:")  
        for label, metrics in report_dict.items():  
            if isinstance(metrics, dict):  # 表示这是类别级别的统计  
                print(f"{label}:")  
                for metric, value in metrics.items():  
                    print(f"  {metric}: {value:.3f}")  
            else:  # 针对"accuracy"这个单值指标  
                print(f"{label}: {metrics:.3f}")
# 示例调用
file1_path = '/root/Schema-Value/data/bird/dataset_new/llama/sql1-result-gpt.json'
file2_path = '/root/Schema-Value/data/bird/dataset_new/llama/sql2-result-gpt.json'
combine_error_types(file1_path, file2_path)


预测为错误的： 1000
实际为错误的： 466
precision: 0.466
              precision    recall  f1-score   support

   incorrect       0.47      0.93      0.62       501
     correct       0.87      0.31      0.46       774

    accuracy                           0.55      1275
   macro avg       0.67      0.62      0.54      1275
weighted avg       0.71      0.55      0.52      1275

Classification Report:
incorrect:
  precision: 0.466
  recall: 0.930
  f1-score: 0.621
  support: 501.000
correct:
  precision: 0.873
  recall: 0.310
  f1-score: 0.458
  support: 774.000
accuracy: 0.554
macro avg:
  precision: 0.669
  recall: 0.620
  f1-score: 0.539
  support: 1275.000
weighted avg:
  precision: 0.713
  recall: 0.554
  f1-score: 0.522
  support: 1275.000


In [2]:
import sqlite3
import multiprocessing


def execute_query(database_name, sql):
    """在子进程中执行 SQL 查询"""
    try:
        conn = sqlite3.connect(database_name)
        cursor = conn.cursor()
        cursor.execute(sql)
        result = cursor.fetchall()
        conn.close()
        return result
    except Exception as e:
        return str(e)


def run_with_timeout(database_name, sql, timeout=100):
    """设置超时执行查询"""
    with multiprocessing.Pool(processes=1) as pool:
        # 使用 apply_async 异步执行查询
        result = pool.apply_async(execute_query, (database_name, sql))
        try:
            return result.get(timeout=timeout)  # 等待超时秒数
        except multiprocessing.TimeoutError:
            pool.terminate()
            return "查询超时"


# 数据库路径
database_name = '/root/hdd/BIRD/data/sft_data_collections/bird/dev/dev_databases/codebase_community/codebase_community.sqlite'
sql1 = 'SELECT u.displayname FROM users u WHERE u.downvotes = 0 AND EXISTS ( SELECT 1 FROM comments c WHERE c.userid = u.id AND c.score BETWEEN 1 AND 5 );'

# 执行 SQL 查询并设置超时
result = run_with_timeout(database_name, sql1, timeout=100)
print("查询结果:", result)


查询结果: 查询超时


In [6]:
import json  
from collections import defaultdict  

def find_duplicates_by_field(file_path, field_name):  
    """  
    找到 JSON 文件中根据指定字段的重复数据。  

    参数:  
    - file_path: JSON 文件路径。  
    - field_name: 用于检测重复的字段名。  

    返回:  
    - 重复项的列表。  
    """  
    with open(file_path, 'r', encoding='utf-8') as f:  
        data = json.load(f)  

    seen = defaultdict(list)  
    
    # 记录所有项目，根据field_name进行分组  
    for item in data:  
        key = item.get(field_name)  
        if key is not None:  
            seen[key].append(item)  

    # 过滤出重复项目（列表长度大于1）  
    duplicates = [items for items in seen.values() if len(items) > 1]  
    
    return duplicates  

if __name__ == "__main__":  
    file_path = '/root/Schema-Value/data/bird/datasets/sql1.json'  
    duplicates = find_duplicates_by_field(file_path, 'question')  

    print(f"找到 {len(duplicates)} 组重复项：")  
    for group in duplicates:  
        for item in group:  
            print(json.dumps(item, ensure_ascii=False, indent=4))

找到 55 组重复项：
{
    "db_id": "card_games",
    "question": "How many card border with black color ? List out the card id. border with black color refers to borderColor = 'black'",
    "pred_sql": "SELECT count(id) FROM cards WHERE bordercolor = 'black'",
    "err_type": "result",
    "full_schema": "database schema: table cards, columns = [isreprint, istimeshifted, keywords, faceconvertedmanacost, edhrecrank, asciiname, isstarter, multiverseid, hand, isstoryspotlight, life, manacost, originaltype, rarity, isalternative, loyalty, frameeffects, types, scryfallid, isoversized, hasnonfoil, mtgarenaid, variations, colorindicator, scryfalloracleid, printings, istextless, facename, isfullart, isreserved, bordercolor, type, id, watermark, name, flavortext, ispromo, originalreleasedate, dueldeck, mtgoid, number, subtypes, frameversion, mtgofoilid, hasfoil, hasalternativedecklimit, mcmmetaid, originaltext, availability, isonlineonly, convertedmanacost, setcode, colors, tcgplayerproductid, leadersh

In [7]:
import json  

def remove_duplicates(file_path, field_name, output_file):  
    """  
    从 JSON 文件中根据指定字段去除重复数据，只保留首次出现的项。  

    参数:  
    - file_path: 输入 JSON 文件路径。  
    - field_name: 用于检测重复的字段名。  
    - output_file: 输出去重后数据的文件路径。  
    """  
    # 读取 JSON 数据  
    with open(file_path, 'r', encoding='utf-8') as f:  
        data = json.load(f)  

    seen = set()  
    unique_data = []  

    # 遍历 JSON 数据，按顺序检查并保留首次出现的项目  
    for item in data:  
        key = item.get(field_name)  
        if key is not None and key not in seen:  
            seen.add(key)  
            unique_data.append(item)  

    # 写入去重后的数据到新文件  
    with open(output_file, 'w', encoding='utf-8') as f:  
        json.dump(unique_data, f, ensure_ascii=False, indent=4)  

if __name__ == "__main__":  
    input_file_path = '/root/Schema-Value/data/bird/datasets/sql1.json'  
    output_file_path = '/root/Schema-Value/data/bird/datasets/sql1_unique.json'  
    remove_duplicates(input_file_path, 'question', output_file_path)  

    print(f"去重后的数据已保存到 {output_file_path}")

去重后的数据已保存到 /root/Schema-Value/data/bird/datasets/sql1_unique.json


多轮

In [21]:
import json
from sklearn.metrics import  classification_report
import numpy as np

def combine_error_types(file1_path, file2_path):
    # 读取JSON文件
    with open(file1_path, 'r', encoding='utf-8') as file1, open(file2_path, 'r', encoding='utf-8') as file2:
        data1 = json.load(file1)
        data2 = json.load(file2)


    # 假设两个文件的数据是一个列表，每个元素对应一个SQL记录
    combined_data = []
    y_true = []
    y_pred = []
    true_err_count=0
    pred_err_count=0
    for item1, item2 in zip(data1, data2):
        # 获取两个文件中的new_err_type
        new_err_type1 = item1.get('new_err_type', '')
        new_err_type2 = item2.get('new_err_type', '')
        true_err_type = item1.get('err_type','')

        # 判断combine_err_type的值
        if new_err_type1 != 'correct' and new_err_type2 != 'correct':
            pred_err_count+=1
            if true_err_type != 'correct':
                true_err_count+=1

        true_label = 0 if true_err_type != "correct" else 1
        # 根据 new_err_type 确定 predicted label
        pred_label = 0 if new_err_type1 != "correct" and new_err_type2!="correct" else 1

        # 仅在不是超时的情况下记录
        # if item['new_err_type'] != "timeout":
        y_true.append(true_label)
        y_pred.append(pred_label)

    print("预测为错误的：",pred_err_count)
    print("实际为错误的：",true_err_count)
    print("precision:", true_err_count*1.0 / pred_err_count)
    # pred_labels = np.array([0]*259)
    # true_labels = np.array([0]*197 + [1]*62)
    # y_true += true_labels.tolist()
    # y_pred += pred_labels.tolist()

    if y_true and y_pred:

        report = classification_report(y_true, y_pred, target_names=["incorrect", "correct"])
        print(report)
        report_dict = classification_report(y_true, y_pred, target_names=["incorrect", "correct"], output_dict=True)  

        # 格式化和打印报告  
        print("Classification Report:")  
        for label, metrics in report_dict.items():  
            if isinstance(metrics, dict):  # 表示这是类别级别的统计  
                print(f"{label}:")  
                for metric, value in metrics.items():  
                    print(f"  {metric}: {value:.3f}")  
            else:  # 针对"accuracy"这个单值指标  
                print(f"{label}: {metrics:.3f}")
# 示例调用
file1 = '/root/Schema-Value/data/bird/dataset_new/deepseek/sql2-1-result.json'
file2 = '/root/Schema-Value/data/bird/dataset_new/OURS-GPT/round2/round2sql1-1-result.json'
combine_error_types(file1, file2)


预测为错误的： 446
实际为错误的： 318
precision: 0.7130044843049327
              precision    recall  f1-score   support

   incorrect       0.71      0.63      0.67       501
     correct       0.78      0.83      0.81       774

    accuracy                           0.76      1275
   macro avg       0.75      0.73      0.74      1275
weighted avg       0.75      0.76      0.75      1275

Classification Report:
incorrect:
  precision: 0.713
  recall: 0.635
  f1-score: 0.672
  support: 501.000
correct:
  precision: 0.779
  recall: 0.835
  f1-score: 0.806
  support: 774.000
accuracy: 0.756
macro avg:
  precision: 0.746
  recall: 0.735
  f1-score: 0.739
  support: 1275.000
weighted avg:
  precision: 0.753
  recall: 0.756
  f1-score: 0.753
  support: 1275.000


多轮交集plus

In [2]:
import json
from sklearn.metrics import  classification_report
import numpy as np

def combine_error_types(file1_path, file2_path, file3_path, file4_path):
    # 读取JSON文件
    with open(file1_path, 'r', encoding='utf-8') as file1, open(file2_path, 'r', encoding='utf-8') as file2,open(file3_path, 'r', encoding='utf-8') as file3,open(file4_path, 'r', encoding='utf-8') as file4:
        data1 = json.load(file1)
        data2 = json.load(file2)
        data3 = json.load(file3)
        data4 = json.load(file4)


    # 假设两个文件的数据是一个列表，每个元素对应一个SQL记录
    combined_data = []
    y_true = []
    y_pred = []
    true_err_count=0
    pred_err_count=0
    for item1, item2,item3,item4 in zip(data1, data2,data3,data4):
        # 获取两个文件中的new_err_type
        new_err_type1 = item1.get('new_err_type', '')
        new_err_type2 = item2.get('new_err_type', '')
        new_err_type3 = item3.get('new_err_type', '')
        new_err_type4 = item4.get('new_err_type', '')
        true_err_type = item1.get('err_type','')

        # 判断combine_err_type的值
        if new_err_type1 != 'correct' and new_err_type2 != 'correct' and new_err_type3!='correct' and new_err_type4!='correct':
            pred_err_count+=1
            if true_err_type != 'correct':
                true_err_count+=1

        true_label = 0 if true_err_type != "correct" else 1
        # 根据 new_err_type 确定 predicted label
        pred_label = 0 if new_err_type1 != "correct" and new_err_type2!="correct" and new_err_type3!='correct' and new_err_type4!='correct' else 1

        # 仅在不是超时的情况下记录
        # if item['new_err_type'] != "timeout":
        y_true.append(true_label)
        y_pred.append(pred_label)

    print("预测为错误的：",pred_err_count)
    print("实际为错误的：",true_err_count)
    print("precision:", true_err_count*1.0 / pred_err_count)
    # pred_labels = np.array([0]*259)
    # true_labels = np.array([0]*197 + [1]*62)
    pred_labels = np.array([0]*127 + [1]*48)
    true_labels = np.array([0]*127 + [1]*48)
    
    y_true += true_labels.tolist()
    y_pred += pred_labels.tolist()

    if y_true and y_pred:

        report = classification_report(y_true, y_pred, target_names=["incorrect", "correct"])
        print(report)
        report_dict = classification_report(y_true, y_pred, target_names=["incorrect", "correct"], output_dict=True)  

        # 格式化和打印报告  
        print("Classification Report:")  
        for label, metrics in report_dict.items():  
            if isinstance(metrics, dict):  # 表示这是类别级别的统计  
                print(f"{label}:")  
                for metric, value in metrics.items():  
                    print(f"  {metric}: {value:.3f}")  
            else:  # 针对"accuracy"这个单值指标  
                print(f"{label}: {metrics:.3f}")
# 第一轮
file1_path = '/root/Schema-Value/data/spider/dataset_new/ours_gpt/sql1-2-result.json'
file2_path = '/root/Schema-Value/data/spider/dataset_new/ours_gpt/sql2-1-result.json'
# 第二轮
file1 = '/root/Schema-Value/data/spider/dataset_new/deepseek/sql1-result.json'
file2 = '/root/Schema-Value/data/spider/dataset_new/deepseek/sql2-result.json'
combine_error_types(file1_path,file2_path,file1, file2)


预测为错误的： 102
实际为错误的： 65
precision: 0.6372549019607843
              precision    recall  f1-score   support

   incorrect       0.84      0.76      0.80       254
     correct       0.92      0.95      0.94       780

    accuracy                           0.90      1034
   macro avg       0.88      0.85      0.87      1034
weighted avg       0.90      0.90      0.90      1034

Classification Report:
incorrect:
  precision: 0.838
  recall: 0.756
  f1-score: 0.795
  support: 254.000
correct:
  precision: 0.923
  recall: 0.953
  f1-score: 0.938
  support: 780.000
accuracy: 0.904
macro avg:
  precision: 0.881
  recall: 0.854
  f1-score: 0.866
  support: 1034.000
weighted avg:
  precision: 0.902
  recall: 0.904
  f1-score: 0.903
  support: 1034.000


In [ ]:
file1='/root/Schema-Value/data/bird/dataset_new/llama/sql1-1-result.json'
file2='/root/Schema-Value/data/bird/dataset_new/llama/sql2-1-result.json'



In [ ]:
import json

# 输入文件路径
file1 = '/root/Schema-Value/data/bird/dataset_new/llama/sql1-1-result.json'
file2 = '/root/Schema-Value/data/bird/dataset_new/llama/sql2-1-result.json'

# 输出文件路径
output_file = '/root/Schema-Value/data/bird/dataset_new/llama/filtered_results_llama.json'

# 读取文件1和文件2的内容
with open(file1, 'r') as f1:
    data1 = json.load(f1)

with open(file2, 'r') as f2:
    data2 = json.load(f2)

# 定义一个过滤函数，确保至少一个文件中的条目 "new_err_type" 是 "correct"
def filter_data(data1, data2):
    filtered_data = []
    # 确保两个文件的条目数量相同
    for entry1, entry2 in zip(data1, data2):
        if entry1.get('new_err_type') != 'correct' and entry2.get('new_err_type') != 'correct':
            continue
        filtered_data.append(entry1)  # 只将符合条件的条目加入
    return filtered_data

# 过滤数据
filtered_data = filter_data(data1, data2)

# 将过滤后的数据写入新文件
with open(output_file, 'w') as output_f:
    json.dump(filtered_data, output_f, indent=4)

print(f"Filtered data has been saved to {output_file}")


Filtered data has been saved to /root/Schema-Value/data/bird/dataset_new/OURS-GPT/round2/filtered_results.json


In [13]:
import json

file0 = '/root/Schema-Value/data/bird/dataset_new/OURS-GPT/round2/round2sql1-1-result.json'
file1 = '/root/Schema-Value/data/bird/dataset_new/OURS-GPT/sql2-result-gpt.json'
file2 = '/root/Schema-Value/data/bird/dataset_new/OURS-GPT/sql1-result-gpt.json'



output_file = '/root/Schema-Value/data/bird/dataset_new/OURS-GPT/round2/filtered_results.json'



# 读取文件1和文件2的内容

with open(file0, 'r') as f0:
    data0 = json.load(f0)
with open(file1, 'r') as f1:
    data1 = json.load(f1)

with open(file2, 'r') as f2:
    data2 = json.load(f2)

# 定义一个过滤函数，确保至少一个文件中的条目 "new_err_type" 是 "correct"
def filter_data(data0,data1, data2):
    filtered_data = []
    # 确保两个文件的条目数量相同
    for entry0, entry1, entry2 in zip(data0, data1, data2):
        if entry1.get('new_err_type') != 'correct' and entry2.get('new_err_type') != 'correct':
            continue
        filtered_data.append(entry0)  # 只将符合条件的条目加入
    return filtered_data

# 过滤数据
filtered_data = filter_data(data0,data1, data2)

# 将过滤后的数据写入新文件
with open(output_file, 'w') as output_f:
    json.dump(filtered_data, output_f, indent=4)

print(f"Filtered data has been saved to {output_file}")


Filtered data has been saved to /root/Schema-Value/data/bird/dataset_new/OURS-GPT/round2/filtered_results.json


第二轮生成的已经和initial比较了，结果筛出来看看数量就可以了

In [17]:
import json
from sklearn.metrics import  classification_report
import numpy as np

def single_error_types(file1_path):
    # 读取JSON文件
    with open(file1_path, 'r', encoding='utf-8') as file1:
        data1 = json.load(file1)


    # 假设两个文件的数据是一个列表，每个元素对应一个SQL记录
    combined_data = []
    y_true = []
    y_pred = []
    true_err_count=0
    pred_err_count=0
    for item1 in data1:
        # 获取两个文件中的new_err_type
        new_err_type1 = item1.get('new_err_type', '')
        true_err_type = item1.get('err_type','')

        # 判断combine_err_type的值
        if new_err_type1 != 'correct' :
            pred_err_count+=1
            if true_err_type != 'correct':
                true_err_count+=1

        true_label = 0 if true_err_type != "correct" else 1
        # 根据 new_err_type 确定 predicted label
        pred_label = 0 if new_err_type1 != "correct"  else 1

        # 仅在不是超时的情况下记录
        # if item['new_err_type'] != "timeout":
        y_true.append(true_label)
        y_pred.append(pred_label)

    print("预测为错误的：",pred_err_count)
    print("实际为错误的：",true_err_count)
    print("precision:", true_err_count*1.0 / pred_err_count)
    # pred_labels = np.array([0]*259)
    # true_labels = np.array([0]*197 + [1]*62)
    # y_true += true_labels.tolist()
    # y_pred += pred_labels.tolist()

    if y_true and y_pred:

        report = classification_report(y_true, y_pred, target_names=["incorrect", "correct"])
        print(report)
        report_dict = classification_report(y_true, y_pred, target_names=["incorrect", "correct"], output_dict=True)  

        # 格式化和打印报告  
        print("Classification Report:")  
        for label, metrics in report_dict.items():  
            if isinstance(metrics, dict):  # 表示这是类别级别的统计  
                print(f"{label}:")  
                for metric, value in metrics.items():  
                    print(f"  {metric}: {value:.3f}")  
            else:  # 针对"accuracy"这个单值指标  
                print(f"{label}: {metrics:.3f}")
# 示例调用
file1 = '/root/Schema-Value/data/bird/dataset_new/llama/sql2-1-result.json'
single_error_types(file1)


预测为错误的： 1066
实际为错误的： 457
precision: 0.42870544090056284
              precision    recall  f1-score   support

   incorrect       0.43      0.91      0.58       501
     correct       0.79      0.21      0.34       774

    accuracy                           0.49      1275
   macro avg       0.61      0.56      0.46      1275
weighted avg       0.65      0.49      0.43      1275

Classification Report:
incorrect:
  precision: 0.429
  recall: 0.912
  f1-score: 0.583
  support: 501.000
correct:
  precision: 0.789
  recall: 0.213
  f1-score: 0.336
  support: 774.000
accuracy: 0.488
macro avg:
  precision: 0.609
  recall: 0.563
  f1-score: 0.459
  support: 1275.000
weighted avg:
  precision: 0.648
  recall: 0.488
  f1-score: 0.433
  support: 1275.000


In [3]:
from openai import OpenAI
instruction='''Answer the question using only an SQLite SQL query, combined with the database schema and SQL skeleton, with no explanation. For column names containing spaces, please strictly escape them as quotation marks.'''

inputs= '''Question: How many schools with an average score in Math greater than 400 in the SAT test are exclusively virtual? Exclusively virtual refers to Virtual = 'F'
Database schema: table schools, columns = [`virtual` (values: 'F'), `school`, `cdscode`]; table satscores, columns = [`avgscrmath`, `cds`]
SQL skeleton: SELECT COUNT() FROM _ s JOIN _ sc ON s._ = sc._ WHERE s._ = _ AND sc._ > _;
SQL:'''
client = OpenAI(api_key="sk-omfgopydsppfscytmsdnsxlqappnvyxynqhqongaokpskeno", base_url="https://api.siliconflow.cn/v1")
response = client.chat.completions.create(
    model='deepseek-ai/DeepSeek-V3',
    messages=[
            {"role": "system", "content": instruction},
            {"role": "user", "content": inputs},
        ],
    stream=False
)
response.choices[0].message.content
# for chunk in response:
#     print(chunk.choices[0].delta.content, end='')

"SELECT COUNT(*) FROM schools s JOIN satscores sc ON s.`cdscode` = sc.`cds` WHERE s.`virtual` = 'F' AND sc.`avgscrmath` > 400;"